In [1]:
all_words = open('dictionary.txt').read().split('\n')
# dictionary.txt is a txt file of all valid words taken from the link on Piazza
# https://raw.githubusercontent.com/dwyl/english-words/master/words_alpha.txt

In [2]:
import pandas as pd
df = pd.read_csv('unigram_freq.csv', index_col = False)
# unigram_freq.csv is a csv file from Kaggle, containing english words and word frequency
# Data is derived from the Google Web Trillion Word Corpus.
# https://www.kaggle.com/rtatman/english-word-frequency

In [3]:
df = df[df['word'].isin(all_words)] # Clean kaggle dataset

In [4]:
word_counter = {}
for i in range(len(df)):
    word_counter[df.iloc[i,0]] = df.iloc[i,1]

In [5]:
for word in all_words:
    if word not in word_counter:
        word_counter[word] = 1

In [6]:
word_counter["I"] = word_counter["i"]
word_counter.pop("i")

3086225277

In [7]:
def spell_corrector(word_list):
    return [correct(word) for word in word_list]

In [8]:
# Corrects word if it's alphanumeric
# Else, do nothing
# Capitalizes it accordingly
def correct(word):
    if not word.isalnum():
        return word
    if word == "I":
        return "I"
    corrected_word = best_candidate(word.lower())
    if word.isupper():
        return corrected_word.upper()
    if word[0].isupper():
        return corrected_word[0].upper() + corrected_word[1:]
    return corrected_word

In [9]:
# Returns best candidate for a given word
# Prioritizes lower Levenshtein distance, and then word frequency
# If there are no words with Levenshtein distance ≤ 2 in word_counter, return the word itself
def best_candidate(word):
    return (best_candidate_from_list([word]) or best_candidate_from_list(distance1(word)) 
            or best_candidate_from_list(distance2(word)) or [word])

In [10]:
# Returns best candidate from a list of words
# Weighted by word frequency
def best_candidate_from_list(words):
    count = -1
    candidate = None
    for w in words:
        if w in word_counter and word_counter[w] > count:
            count = word_counter[w]
            candidate = w
    return candidate

In [11]:
# Set of words of one Levenshtein distance from word
def distance1(word):
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    ret = set()
    for i in range(len(word)):
        ret.add(word[:i] + word[i+1:]) #Deletion
        for letter in letters:
            ret.add(word[:i] + letter + word[i:]) #Insertion
            ret.add(word[:i] + letter + word[i+1:]) #Substitution
    return ret

In [12]:
# Set of words of two Levenshtein distance from word
def distance2(word): 
    ret = set()
    for distance1_word in distance1(word):
        ret.update(distance1(distance1_word))
    return ret

In [13]:
sentence = ["CHAPOTER", " ", "1", "\n", "\n", "\n", "The", "family", "of", "Dashwood", "had", "long", "been", "settled", "i", "Sussex."]
spell_corrector(sentence)

['CHAPTER',
 ' ',
 'a',
 '\n',
 '\n',
 '\n',
 'The',
 'family',
 'of',
 'Basswood',
 'had',
 'long',
 'been',
 'settled',
 'a',
 'Sussex.']